In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime

# 1. GRAPHICAL CONFIGURATION
# Set figure size and use "Agg" backend to ensure compatibility in all environments
plt.rcParams["figure.figsize"] = (10, 6)
import matplotlib
matplotlib.use("Agg")

# 2. PARAMETERS
TICKERS = ["TSLA", "JPM", "COST", "LLY"]
MARKET  = "^GSPC"  # S&P 500 Index
RF_RATE = "^IRX"    # Risk-Free Rate (13-week Treasury Bill)
START   = "2023-01-01"
END     = "2025-01-01"

def run_advanced_capm_analysis():
    print(f"Downloading data for: {TICKERS} + {MARKET} + {RF_RATE}")

    # 3. DATA ACQUISITION
    all_tickers = TICKERS + [MARKET, RF_RATE]
    data = yf.download(all_tickers, start=START, end=END, auto_adjust=True, progress=False)['Close']

    # Clean data: Forward fill missing values and drop remaining NaNs
    data = data.ffill().dropna()

    # 4. CALCULATE DAILY RETURNS
    # Converts prices into percentage change
    returns = data[TICKERS + [MARKET]].pct_change().dropna()

    # 5. RISK-FREE RATE PROCESSING
    # Convert annual Treasury Bill yield to daily yield
    rf_annual = data[RF_RATE].reindex(returns.index).ffill() / 100
    rf_daily = (1 + rf_annual)**(1/252) - 1

    # 6. MARKET EXCESS RETURN (Rm - Rf)
    market_excess = returns[MARKET] - rf_daily

    summary_results = []

    # 7. ANALYSIS LOOP PER TICKER
    for stock in TICKERS:
        # Stock Excess Return (Ri - Rf)
        stock_excess = returns[stock] - rf_daily

        # OLS Regression Setup (Adding a constant for the Alpha intercept)
        X = sm.add_constant(market_excess)
        model = sm.OLS(stock_excess, X).fit()

        # SECURE EXTRACTION (Using .iloc to avoid label-based KeyErrors)
        # .iloc[0] = Constant (Alpha) | .iloc[1] = Slope (Beta)
        alpha_daily = model.params.iloc[0]
        beta = model.params.iloc[1]
        r_squared = model.rsquared
        p_value_alpha = model.pvalues.iloc[0]

        # Store results in a dictionary
        summary_results.append({
            "Ticker": stock,
            "Beta (Risk)": round(beta, 3),
            "Alpha Annual (%)": round(alpha_daily * 252 * 100, 2),
            "R-Squared": round(r_squared, 3),
            "Alpha Significant?": "Yes" if p_value_alpha < 0.05 else "No"
        })

        # 8. PLOT GENERATION
        plt.figure()
        plt.scatter(market_excess, stock_excess, alpha=0.5, s=15, label="Daily Observations")
        plt.plot(market_excess, alpha_daily + beta * market_excess, color='red', lw=2, label=f"CAPM Line (Beta: {beta:.2f})")
        plt.title(f"CAPM Regression Analysis: {stock}")
        plt.xlabel("Market Excess Return (S&P 500)")
        plt.ylabel(f"{stock} Excess Return")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f"capm_reg_{stock}.png")
        plt.close()

    # 9. FINAL REPORTING
    df_results = pd.DataFrame(summary_results).set_index("Ticker")
    df_results.to_csv("capm_advanced_report.csv")

    print("\n--- ADVANCED CAPM REPORT ---")
    print(df_results)
    print("\nCharts (.png) and the CSV report have been successfully generated.")

if __name__ == "__main__":
    run_advanced_capm_analysis()


--- ADVANCED CAPM REPORT ---
        Beta (Risk)  Alpha Annual (%)  R-Squared Alpha Significant?
Ticker                                                             
TSLA          2.299             37.73      0.257                 No
JPM           0.841             14.27      0.238                 No
COST          0.822             19.68      0.311                 No
LLY           0.649             26.52      0.080                 No

Charts (.png) and the CSV report have been successfully generated.
